### Planning Algorithms

In [1]:
import numpy as np
import pprint
import json
import sys

#### Policy Evaluation

In [2]:
def policy_evaluation(policy, env, discount_factor=.75, theta=0.000001):
    V = np.zeros(env.nS)
    # print('evaluating policy')
    while True:
        delta = 0
        # For each state, perform a "full backup"
        for s in range(env.nS):
            v = 0
            # Look at the possible next actions
            for a, action_prob in enumerate(policy[s]):
                # For each action, look at the possible next states...
                for  prob, next_state, reward, done in env.P[s][a]:
                    # Calculate the expected value
                    v += action_prob * prob * (reward + discount_factor * V[next_state] * (not done))
                    # print(s, a, prob, next_state, reward, done, v)
            # How much our value function changed (across any states)
            delta = max(delta, np.abs(v - V[s]))
            # print(delta)
            V[s] = v
        # Stop evaluating once our value function change is below a threshold
        if delta < theta:
            break
    return np.array(V)

#### Policy Improvement

In [3]:
def policy_improvement(env, policy_eval_fn=policy_evaluation, discount_factor=0.75):
    # Start with a random policy
    policy = np.random.random([env.nS, env.nA])

    iterations = 0
    while True:
        # Evaluate the current policy
        V = policy_eval_fn(policy, env, discount_factor)
        iterations += 1
        # Will be set to false if we make any changes to the policy
        policy_stable = True
        # For each state...
        for s in range(env.nS):
            # The best action we would take under the currect policy
            chosen_a = np.argmax(policy[s])

            # Find the best action by one-step lookahead
            # Ties are resolved arbitarily
            action_values = np.zeros(env.nA)
            for a in range(env.nA):
                for prob, next_state, reward, done in env.P[s][a]:
                    action_values[a] += prob * (reward + discount_factor * V[next_state])
            best_a = np.argmax(action_values)

            # Greedily update the policy
            if chosen_a != best_a:
                policy_stable = False
            policy[s] = np.eye(env.nA)[best_a]

        # If the policy is stable we've found an optimal policy. Return it
        if policy_stable:
            return policy, V, iterations

#### Policy Iteration

#### Value Iteration

In [4]:
import gym
import numpy as np

from gym import wrappers

def iterate_value_function(v_inp, q_inp, gamma, env):
    
    nstates = env.env.observation_space.n
    nactions = env.env.action_space.n
    
    ret = np.zeros(nstates)
    retq = np.zeros((nstates, nactions))

    for sid in range(nstates):
        temp_v = np.zeros(nactions)
        for action in range(nactions):
            for (prob, dst_state, reward, is_final) in env.env.env.P[sid][action]:
                if is_final:
                    temp_v[action] = temp_v[action] + prob * (reward)
                else:
                    temp_v[action] = temp_v[action] + prob * (reward + gamma * v_inp[dst_state])
        retq[sid] = temp_v
        ret[sid] = max(temp_v)
    return ret, retq

def build_greedy_policy(v_inp, gamma, env):
    nstates = env.env.observation_space.n
    nactions = env.env.action_space.n
    
    new_policy = np.zeros(nstates)
    
    for state_id in range(nstates):
        profits = np.zeros(nactions)
        for action in range(nactions):
            for (prob, dst_state, reward, is_final) in env.env.env.P[state_id][action]:
                profits[action] += prob*(reward + gamma*v[dst_state])
        new_policy[state_id] = np.argmax(profits)
    return new_policy


env = gym.make('Taxi-v2')
env = wrappers.Monitor(env, '/tmp/', force=True)

gamma = 0.9
cum_reward = 0
n_rounds = 1000

# solve the world with value iteration 
# (without interacting with the environment)

for t_rounds in range(n_rounds):
    # init env and value function
    observation = env.reset()
    
    v = np.zeros(env.env.observation_space.n)
    q = np.zeros((env.env.observation_space.n, 
                  env.env.action_space.n))

    # solve MDP
    for _ in range(200):
        v_old = v.copy()
        q_old = q.copy()
        v, q = iterate_value_function(v, q, gamma, env)
        if np.all(v == v_old):
            break
    policy = build_greedy_policy(v, gamma, env).astype(np.int)

    # apply policy
    for t in range(1000):
        action = policy[observation]
        observation, reward, done, info = env.step(action)
        cum_reward += reward
        if done:
            break
    if t_rounds % 50 == 0 and t_rounds > 0:
        print(cum_reward * 1.0 / (t_rounds + 1))

np.set_printoptions(threshold=np.nan)
print('v states')
#print(v.tolist())
print('q values')
#print(q.tolist())


[2017-03-20 16:55:52,983] Making new env: Taxi-v2
[2017-03-20 16:55:52,992] Clearing 20 monitor files from previous run (because force=True was provided)
[2017-03-20 16:55:52,993] Starting new video recorder writing to /tmp/openaigym.video.0.5991.video000000.json
[2017-03-20 16:55:53,065] Starting new video recorder writing to /tmp/openaigym.video.0.5991.video000001.json
[2017-03-20 16:55:53,538] Starting new video recorder writing to /tmp/openaigym.video.0.5991.video000008.json
[2017-03-20 16:55:54,832] Starting new video recorder writing to /tmp/openaigym.video.0.5991.video000027.json


8.372549019607844


[2017-03-20 16:55:57,342] Starting new video recorder writing to /tmp/openaigym.video.0.5991.video000064.json


8.673267326732674


[2017-03-20 16:56:01,468] Starting new video recorder writing to /tmp/openaigym.video.0.5991.video000125.json


...

8.52991452991453
8.481296758104738
8.467849223946786
8.463073852295409


[2017-03-20 16:56:27,595] Starting new video recorder writing to /tmp/openaigym.video.0.5991.video000512.json


8.450090744101633
8.43926788685524
8.428571428571429
8.455064194008559


[2017-03-20 16:56:42,185] Starting new video recorder writing to /tmp/openaigym.video.0.5991.video000729.json


8.474034620505993
8.460674157303371
8.474735605170387
8.452830188679245
8.43217665615142
v states
q values


In [5]:
# act greedily
for episode in range(10):
    state = env.reset()

    while True:
        action = np.argmax(q[state])
        _, _, done, _ = env.step(action)
        if done:
            break

[2017-03-20 16:57:00,439] Starting new video recorder writing to /tmp/openaigym.video.0.5991.video001000.json


In [ ]:
env.videos[-1][0]

In [ ]:
from subprocess import check_output

out = check_output(["asciinema", "upload", env.videos[-1][0]])
out = out.decode("utf-8").replace('\n', '').replace('\r', '')

In [ ]:
print(out)

In [ ]:
castid = out.split('/')[-1]
castid

In [ ]:
from IPython.display import HTML

html_tag = """
<script type="text/javascript" 
    src="https://asciinema.org/a/{0}.js" 
    id="asciicast-{0}" 
    async data-autoplay="true" data-size="big">
</script>
"""
html_tag = html_tag.format(castid)
HTML(data=html_tag)